# Database connection test

# Preparation

In [1]:
# Imports
import sqlite3
import pandas as pd

## Main connection

In [2]:
# Connect to main database
db_path = '../TsakonianDB.sqlite3'
conn = sqlite3.connect(db_path)

## Auxiliary functions

In [3]:
def query(query: str,
          cursor = conn.cursor()):

      cursor.execute(query)
      fetched = cursor.fetchall()

      if len(fetched) > 0:
            temp_df = pd.DataFrame(fetched, columns=[i[0] for i in cursor.description])
            return temp_df
      else:
            print('Query executed successfully. No results to show.')

# Add words from main dictionary to the database

## Load dictionary

In [10]:
# Load main dictionary
main_df_path = '../../data/tables/main.xlsx'
main_df = pd.read_excel(main_df_path)
main_df

,tsakonian,greek,paradigm,source_id
0,άγο,άλογο,Α0,1
1,άγουστε,αύγουστος,NaN,1
2,άθρωπο,άνθροπος,Α1,1
3,άι,λάδι,Υ0,1
4,άλητε,αλεύρι,Υ1,1
...,...,...,...,...
588,λεκό,λευκός,Ε,1
589,βρέο,οβριός,NaN,1
590,καφέ,καφές,Α,1
591,πέτσ̌ε,πέτρα,NaN,1


## Manage duplicates

In [11]:
# Remove duplicates with the exact same information
print(f'Entries before removing duplicates: {len(main_df)}')
main_df = main_df.drop_duplicates()
print(f'Entries after removing duplicates: {len(main_df)}')

Entries before removing duplicates: 593
Entries after removing duplicates: 590


In [12]:
# Show duplicates
main_df[main_df.duplicated(subset = 'tsakonian', keep = False)].sort_values(by = 'tsakonian')

,tsakonian,greek,paradigm,source_id
22,έτρου,έτσι,Ζ,1
530,έτρου,έτσι,NaN,1


In [13]:
# Keep the duplicate with the lowest ID
print(f'Entries before removing duplicates: {len(main_df)}')
main_df = (main_df
           .sort_index()
           .drop_duplicates(subset = 'tsakonian', keep = 'first')
           .sort_values(by = 'tsakonian')
           .reset_index(drop = True)
           )
print(f'Entries after removing duplicates: {len(main_df)}')
main_df

Entries before removing duplicates: 590
Entries after removing duplicates: 589


,tsakonian,greek,paradigm,source_id
0,άγο,άλογο,Α0,1
1,άγουστε,αύγουστος,NaN,1
2,άζ̌α,"αραία, όχι συχνά",NaN,1
3,άθρωπο,άνθροπος,Α1,1
4,άι,λάδι,Υ0,1
...,...,...,...,...
584,όνε,γάιδαρος,Α3,1
585,όρεγι,εδώ ακριβώς,Ζ,1
586,όρπα,εκεί,Ζ,1
587,ότσι,ότι,Ζ,1


## Save to main Excel

In [14]:
# Save main_df back to the Excel file
main_df.to_excel(main_df_path, index = False)
print('Changes consolidated.')

Changes consolidated.


In [15]:
# Load main dictionary
main_df = pd.read_excel(main_df_path)

# Copy the full dictionary into the database
main_df.to_sql('dictionary_entry', 
               conn, 
               if_exists='replace', 
               index=False,
               dtype = {'tsakonian': 'varchar(50)',
                        'greek': 'varchar(200)',
                        'paradigm': 'varchar(5)',
                        'source_id': 'bigint',
               })

# Delete the temporary table if it exists
try:
    query("DROP TABLE sqlitestudio_temp_table;")
except:
    pass

# Add primary keys and foreign keys by recreating the table
# Obtained from SQLiteStudio
recreating_query = """PRAGMA foreign_keys = 0;

CREATE TABLE sqlitestudio_temp_table AS SELECT *
                                          FROM dictionary_entry;

DROP TABLE dictionary_entry;

CREATE TABLE dictionary_entry (
    tsakonian VARCHAR (50)  PRIMARY KEY
                            NOT NULL,
    greek     VARCHAR (200),
    paradigm  VARCHAR (5),
    source_id    INTEGER       REFERENCES dictionary_source (id) 
);

INSERT INTO dictionary_entry (
                                 tsakonian,
                                 greek,
                                 paradigm,
                                 source_id
                             )
                             SELECT tsakonian,
                                    greek,
                                    paradigm,
                                    source_id
                               FROM sqlitestudio_temp_table;

DROP TABLE sqlitestudio_temp_table;

PRAGMA foreign_keys = 1;"""

# Execute queries in a loop
# Only one query can be executed at a time
for q in recreating_query.split(';'):
      query(q)

# Save changes
conn.commit()

Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
